<a href="https://colab.research.google.com/github/Cloud2602/gesture_recognition/blob/main/gesture_recognition_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Scaricare e aprire acquisizioni.zip
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/acquisizioni.zip" -d "/content/dataset"

Mounted at /content/drive


In [6]:
%%bash
pip install mediapipe

In [ ]:
import os
import numpy as np
import cv2
import mediapipe as mp
import csv

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
)

# Percorsi
DATASET_DIR = "/content/3"  # Cambia se necessario
output_npy = "landmarks.npy"
output_labels = "labels.npy"
output_csv = "landmarks.csv"

# Output
all_landmarks = []
all_labels = []

# Mapping label
label_map = {
    "1": 1,
    "2": 2,
    "tre": 3,
    "0": 0
}

# CSV setup
csv_file = open(output_csv, mode='w', newline='')
csv_writer = csv.writer(csv_file)

# Header
header = [f'{dim}{i}' for i in range(21) for dim in ['x', 'y', 'z']]
header.append('label')
csv_writer.writerow(header)

# Contatori
total_images = 0
processed_images = 0
skipped_images = 0

# Scansiona le sottocartelle
for class_name in sorted(os.listdir(DATASET_DIR)):
    print(f"✔️ Cartella: {class_name}")
    class_path = os.path.join(DATASET_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    if class_name not in label_map:
        print(f"⚠️ La cartella '{class_name}' non ha una label definita. Saltata.")
        continue
    label = label_map[class_name]

    for filename in os.listdir(class_path):
        print(filename)
        if not filename.endswith('_Color.png'):
            continue  # ⛔ Salta file che non terminano in _Color.png

        total_images += 1

        image_path = os.path.join(class_path, filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"❌ Errore nel caricamento di: {image_path}")
            skipped_images += 1
            continue

        # Converti BGR in RGB per MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            hand = results.multi_hand_landmarks[0]
            landmarks = []
            for lm in hand.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            all_landmarks.append(landmarks)
            all_labels.append(label)
            csv_writer.writerow(landmarks + [label])
            processed_images += 1
        else:
            print(f"🙈 Nessuna mano rilevata in: {image_path}")
            skipped_images += 1

# Cleanup
csv_file.close()
hands.close()

np.save(output_npy, np.array(all_landmarks))
np.save(output_labels, np.array(all_labels))

# Report finale
print(f"\n✅ Salvati: {output_npy}, {output_labels}, {output_csv}")
print(f"📊 Totale immagini trovate: {total_images}")
print(f"👌 Immagini elaborate con successo: {processed_images}")
print(f"🚫 Immagini scartate (errore o mano non rilevata): {skipped_images}")


In [1]:
!pip uninstall -y mediapipe
!pip install mediapipe==0.10.9

Found existing installation: mediapipe 0.10.9
Uninstalling mediapipe-0.10.9:
  Successfully uninstalled mediapipe-0.10.9
  Using cached mediapipe-0.10.9-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached mediapipe-0.10.9-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)


In [12]:
!unzip -q "/content/dataset-hands.zip" -d "/content/dataset_hands"

In [21]:
import os
import cv2
import mediapipe as mp
import numpy as np
import csv

# Percorsi
DATASET_DIR = "/content/cartella_immagini"  # Sostituisci con il percorso corretto
output_csv = "landmarks_duemani.csv"
output_npy = "landmarks_duemani.npy"

# Setup MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

# Output
all_landmarks = []
# Header aggiornato: mano destra da x0 a z20, mano sinistra da x0_L a z20_L
header = []

# Mano destra
header += [f'{dim}{i}_R' for i in range(21) for dim in ['x', 'y', 'z']]
# Mano sinistra
header += [f'{dim}{i}_L' for i in range(21) for dim in ['x', 'y', 'z']]

with open(output_csv, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(header)

    for filename in sorted(os.listdir(DATASET_DIR)):
        if not filename.endswith("_Color.png"):
            continue

        image_path = os.path.join(DATASET_DIR, filename)
        image_bgr = cv2.imread(image_path)
        if image_bgr is None:
            print(f"❌ Errore caricamento: {filename}")
            continue

        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        # Landmark inizializzati a zero
        right_hand = [0.0] * (21 * 3)
        left_hand = [0.0] * (21 * 3)

        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                label = handedness.classification[0].label  # "Left" o "Right"
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
                if label == "Right":
                    right_hand = landmarks
                else:
                    left_hand = landmarks

        # Righe composte da mano destra + mano sinistra
        combined = right_hand + left_hand
        all_landmarks.append(combined)
        csv_writer.writerow(combined)

# Salvataggio finale
np.save(output_npy, np.array(all_landmarks))
print(f"✅ Landmark salvati in {output_csv} e {output_npy}")


Image path:/content/dataset_hands/dataset-hands/1/57_Color.png , landmarks: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26262858510017395, 0.6722155809402466, 3.792995073581551e-08, 0.2760705351829529, 0.6350207328796387, -0.016598574817180634, 0.27879640460014343, 0.5887834429740906, -0.02622239477932453, 0.27215954661369324, 0.5484323501586914, -0.03615563362836838, 0.25154033303260803, 0.525679886341095, -0.045297350734472275, 0.26143282651901245, 0.523405134677887, -0.011440753005445004, 0.2551615536212921, 0.4600355327129364, -0.026482978835701942, 0.25073596835136414, 0.4185112714767456, -0.03590070456266403, 0.24391047656536102, 0.3858872056007385, -0.042232003062963486, 0.23941870033740997, 0.5372450947761536